# BPE(Byte Pair Encoding)

- 입력: 문자 단위 코퍼스 (예: `l o w e s t`)

- 과정:

    1. 문자쌍 빈도를 셈 (예: ("l","o"): 3회)

    2. 가장 많이 등장한 쌍을 병합 (`lo`)

    3. 다시 통계 → 병합 반복

- 결과: 점점 긴 subword 단위가 만들어짐 (`low`, `lowest` 등)

In [1]:
word = "lower"

chars = list(word)
chars

['l', 'o', 'w', 'e', 'r']

In [4]:
joined = " ".join(chars)
joined

'l o w e r'

In [5]:
from collections import defaultdict

In [23]:
sentences = [
    "low lower lowest",
    "newer wider",
    "low low low"
]

In [ ]:
# 1️⃣ 단어별로 쪼개기 (BPE는 보통 단어 단위로 학습)
corpus = []
for line in sentences:
    for word in line.split():
        corpus.append(list(word) + ["</w>"])  # 끝 표시(EOW)

corpus

[['l', 'o', 'w', '</w>'],
 ['l', 'o', 'w', 'e', 'r', '</w>'],
 ['l', 'o', 'w', 'e', 's', 't', '</w>'],
 ['n', 'e', 'w', 'e', 'r', '</w>'],
 ['w', 'i', 'd', 'e', 'r', '</w>'],
 ['l', 'o', 'w', '</w>'],
 ['l', 'o', 'w', '</w>'],
 ['l', 'o', 'w', '</w>']]

In [31]:
print("🔹 단어들을 문자 리스트로 쪼갠 결과:")
for w in corpus:
    print(w)

🔹 단어들을 문자 리스트로 쪼갠 결과:
['l', 'o', 'w', '</w>']
['l', 'o', 'w', 'e', 'r', '</w>']
['l', 'o', 'w', 'e', 's', 't', '</w>']
['n', 'e', 'w', 'e', 'r', '</w>']
['w', 'i', 'd', 'e', 'r', '</w>']
['l', 'o', 'w', '</w>']
['l', 'o', 'w', '</w>']
['l', 'o', 'w', '</w>']


In [34]:
def get_pair_stats(corpus):
    pairs = defaultdict(int)
    for symbols in corpus:
        for a, b in zip(symbols, symbols[1:]):
            pairs[(a, b)] += 1
    return pairs

pairs = get_pair_stats(corpus)

print("\n🔹 문자쌍 등장 횟수:")
for p, c in sorted(pairs.items(), key=lambda x: -x[1])[:10]:
    print(f"{p}: {c}")



🔹 문자쌍 등장 횟수:
('l', 'o'): 6
('o', 'w'): 6
('w', '</w>'): 4
('w', 'e'): 3
('e', 'r'): 3
('r', '</w>'): 3
('e', 's'): 1
('s', 't'): 1
('t', '</w>'): 1
('n', 'e'): 1


In [35]:
from collections import defaultdict

# (다시) 문자쌍 빈도
def get_pair_stats(corpus):
    pairs = defaultdict(int)
    for symbols in corpus:                         # symbols: ['l','o','w','</w>'] 같은 리스트
        for a, b in zip(symbols, symbols[1:]):     # 인접 쌍만
            pairs[(a, b)] += 1
    return pairs

# ✅ 핵심: 가장 많이 나온 쌍 (A,B)을 한 단어 안에서 모두 "AB"로 합쳐주는 함수
def merge_once(corpus, pair):
    A, B = pair
    merged = []
    for symbols in corpus:
        out = []
        i = 0
        while i < len(symbols):
            # 바로 옆이 (A,B)라면 합쳐서 한 토큰으로 넣고 2칸 전진
            if i < len(symbols)-1 and symbols[i] == A and symbols[i+1] == B:
                out.append(A + B)
                i += 2
            else:
                out.append(symbols[i])
                i += 1
        merged.append(out)
    return merged

# ---- 학습(여러 번 병합) ----
# 예시 코퍼스 (앞 단계에서 쓰신 것 그대로면 재사용하셔도 됩니다)
sentences = [
    "low lower lowest",
    "newer wider",
    "low low low"
]

# 단어를 문자 리스트로(끝표시 '</w>' 포함)
corpus = []
for line in sentences:
    for word in line.split():
        corpus.append(list(word) + ["</w>"])

# 병합 규칙을 여기에 저장
merges = []

# N번 반복하며 가장 빈도 높은 쌍을 계속 병합
N = 10
for step in range(1, N+1):
    stats = get_pair_stats(corpus)
    if not stats: 
        break
    best = max(stats.items(), key=lambda x: x[1])[0]   # 가장 많이 나온 쌍
    merges.append(best)
    corpus = merge_once(corpus, best)
    print(f"[{step}] merge {best}")
    # 참고: 중간 결과 한두 단어만 찍어보기
    print("   sample:", corpus[0])


[1] merge ('l', 'o')
   sample: ['lo', 'w', '</w>']
[2] merge ('lo', 'w')
   sample: ['low', '</w>']
[3] merge ('low', '</w>')
   sample: ['low</w>']
[4] merge ('e', 'r')
   sample: ['low</w>']
[5] merge ('er', '</w>')
   sample: ['low</w>']
[6] merge ('low', 'er</w>')
   sample: ['low</w>']
[7] merge ('low', 'e')
   sample: ['low</w>']
[8] merge ('lowe', 's')
   sample: ['low</w>']
[9] merge ('lowes', 't')
   sample: ['low</w>']
[10] merge ('lowest', '</w>')
   sample: ['low</w>']


In [36]:
def bpe_tokenize(word, merges):
    # 단어를 문자 + '</w>'로 시작
    symbols = list(word) + ["</w>"]
    # 학습된 merge 규칙을 순서대로 적용
    for A, B in merges:
        i = 0
        out = []
        while i < len(symbols):
            if i < len(symbols)-1 and symbols[i] == A and symbols[i+1] == B:
                out.append(A + B)
                i += 2
            else:
                out.append(symbols[i])
                i += 1
        symbols = out
    # 마지막 끝표시는 제거
    if symbols and symbols[-1] == "</w>":
        symbols = symbols[:-1]
    return symbols

# 테스트
print("lower  ->", bpe_tokenize("lower", merges))
print("lowest ->", bpe_tokenize("lowest", merges))
print("newest ->", bpe_tokenize("newest", merges))


lower  -> ['lower</w>']
lowest -> ['lowest</w>']
newest -> ['n', 'e', 'w', 'e', 's', 't']
